In [25]:
import os
import numpy
import time
from PIL import Image

In [26]:
class parameter:
    def __init__(self):
        self.input_nodes=32*64
        self.layer=3
        self.layer_nodes=[256,16,1]
        self.learning_rate=0.005
        self.activation_function_name='leaky_ReLU'
        self.epoch=10
        self.batchsize=4
        self.loss_function_name='cross_entropy'
    def show(self):
        print('input_nodes=',self.input_nodes)
        print('layer=',self.layer)
        print('layer_nodes=',self.layer_nodes)
        print('learning_rate=',self.learning_rate)
        print('activation_function_name=',self.activation_function_name)
        print()
        print('epoch=',self.epoch)
        print('batchsize=',self.batchsize)
        print('loss_function_name=',self.loss_function_name)
        print('')

In [27]:
def activation_function(x,activationfunctionname):
    if activationfunctionname=='sigmoid':
        return 1/(1+numpy.exp(-x))
    if activationfunctionname=='tanh':
        return (numpy.exp(x)-numpy.exp(-x))/(numpy.exp(x)+numpy.exp(-x))
    if activationfunctionname=='ReLU':
        return numpy.maximum(0,x)
    if activationfunctionname=='leaky_ReLU':
        return numpy.maximum(0.01*x,x)

In [28]:
def activation_prime(x,activationfunctionname):
    if activationfunctionname=='sigmoid':
        return activation_function(x,'sigmoid')*(1-activation_function(x,'sigmoid'))
    if activationfunctionname=='tanh':
        return 1-activation_function(x,'tanh')*activation_function(x,'tanh')
    if activationfunctionname=='ReLU':
        x[x>=0]=1
        x[x<0]=0
        return x
    if activationfunctionname=='leaky_ReLU':
        x[x>=0]=1
        x[x<0]=0.01
        return x

In [29]:
def loss_function(y,yt,lossfunctionname):
    if lossfunctionname=='cross_entropy':
        return -(yt*numpy.log(y)+(1-yt)*numpy.log(1-y))
    if lossfunctionname=='squared':
        return (y-yt)*(y-yt)/2

In [30]:
class NeuralNetwork:
    def __init__(self,parameter):
        self.input_nodes=parameter.input_nodes
        self.layer=parameter.layer
        self.layer_nodes=parameter.layer_nodes
        self.w=[numpy.random.normal(0,1,(self.layer_nodes[0],self.input_nodes))]
        self.b=[numpy.array(numpy.random.normal(0,1,(self.layer_nodes[0],1)),ndmin=2)]
        self.mean=numpy.zeros((self.input_nodes,1))
        self.var=numpy.ones((self.input_nodes,1))
        for i in range(1,self.layer):
            self.w.append(numpy.random.normal(0,1,(self.layer_nodes[i],self.layer_nodes[i-1])))
            self.b.append(numpy.array(numpy.random.normal(0,1,(self.layer_nodes[i],1)),ndmin=2))
        self.activation_function_name=parameter.activation_function_name
    def forward_propagation(self,inputlist):
        outputs=numpy.array(inputlist,ndmin=2)
        outputs=(outputs-self.mean)/numpy.sqrt(self.var+0.01)
        for i in range(0,self.layer-1):
            inputs=numpy.dot(self.w[i],outputs)+numpy.array(self.b[i],ndmin=2).reshape(-1,1)
            outputs=activation_function(inputs,self.activation_function_name)
        inputs=numpy.dot(self.w[self.layer-1],outputs)+numpy.array(self.b[self.layer-1],ndmin=2).reshape(-1,1)
        outputs=activation_function(inputs,'sigmoid')
        return outputs

In [31]:
def error_test(nn,test_x,test_y,parameter):
    batchsize=parameter.batchsize
    num_batch=int(test_x.shape[1]/batchsize)
    total_num=num_batch*batchsize
    error=0
    e0=0
    e1=0
    t0=0
    for i in range(0,num_batch):
        x=test_x[:,i*batchsize:(i+1)*batchsize]
        yt=test_y[:,i*batchsize:(i+1)*batchsize]
        y=nn.forward_propagation(x)
        for j in range(0,batchsize):
            if y[0][j]<0.5:
                y[0][j]=0
            elif y[0][j]>0.5:
                y[0][j]=1
        dy=y-yt            
        for j in range(0,batchsize):
            if yt[0][j]==0:
                t0+=1
            if dy[0][j]!=0:
                error=error+1
                if yt[0][j]==0:
                    e0+=1
                else:
                    e1+=1
    print('背景误判率',round(100*e0/t0,2),'%:（',e0,'/',t0,'）  对象误判率',round(100*e1/(total_num-t0),2),'%:（',e1,'/',total_num-t0,'）')
    print(total_num,'个测试样本中错误',error,'个，错误率为',round(100*error/total_num,2),'%.')
    return 100*error/total_num

In [32]:
def train(nn,train_x,train_y,test_x,test_y,parameter):
    batchsize=parameter.batchsize
    num_batch=int(train_x.shape[1]/batchsize)
    for i in range(0,nn.input_nodes):
        nn.mean[i]=numpy.mean(train_x[i])
        nn.var[i]=numpy.var(train_x[i])
    for i in range(0,parameter.epoch):
        print('开始训练第 ',i+1,' 轮,每轮 ',num_batch,'批')
        cost_time=time.time()
        for j in range(0,num_batch):
            trainx=train_x[:,j*batchsize:(j+1)*batchsize]
            trianx=(trainx-nn.mean)/numpy.sqrt(nn.var+0.01)
            trainy=train_y[:,j*batchsize:(j+1)*batchsize]
            if parameter.loss_function_name=='cross_entropy':
                z=[numpy.dot(nn.w[0],trainx)+numpy.array(nn.b[0],ndmin=2).reshape(-1,1)]
                a=[activation_function(z[0],nn.activation_function_name)]
                for k in range(1,nn.layer-1):
                    z.append(numpy.dot(nn.w[k],a[k-1])+numpy.array(nn.b[k],ndmin=2).reshape(-1,1))
                    a.append(activation_function(z[k],nn.activation_function_name))
                z.append(numpy.dot(nn.w[nn.layer-1],a[nn.layer-2])+nn.b[nn.layer-1])
                a.append(activation_function(z[nn.layer-1],'sigmoid'))
            else:
                print('没有此损失函数的反向计算')
                return
            dz=[a[nn.layer-1]-trainy]
            dw=[numpy.dot(dz[0],a[nn.layer-2].T)/batchsize]
            db=[numpy.sum(dz[0],axis=1,keepdims=True)]
            for k in range(1,nn.layer-1):
                dz.append(numpy.dot(nn.w[nn.layer-k].T,dz[k-1])*activation_prime(z[nn.layer-k-1],nn.activation_function_name))
                dw.append(numpy.dot(dz[k],a[nn.layer-k-2].T)/batchsize)
                db.append(numpy.sum(dz[k],axis=1,keepdims=True))
            dz.append(numpy.dot(nn.w[1].T,dz[nn.layer-2])*activation_prime(z[0],nn.activation_function_name))
            dw.append(numpy.dot(dz[nn.layer-1],trainx.T)/batchsize)
            db.append(numpy.sum(dz[nn.layer-1],axis=1,keepdims=True))
            for k in range(0,nn.layer-1):
                nn.w[k]-=parameter.learning_rate*dw[nn.layer-k-1]
                nn.b[k]=numpy.array(nn.b[k],ndmin=2).reshape(-1,1)-parameter.learning_rate*db[nn.layer-k-1]
        cost_time=time.time()-cost_time
        print('已完成',i+1,'/',parameter.epoch,' 本轮用时',round(cost_time,2),'秒')
        error_test(nn,test_x,test_y,parameter)
    return nn

In [33]:
def savenn(nn,file_name):
    os.mkdir(file_name)
    para=numpy.array([nn.input_nodes,nn.layer])
    numpy.savetxt(file_name+'/parameter',para)
    layer=numpy.array(nn.layer_nodes)
    numpy.savetxt(file_name+'/layer',layer)
    f=open(file_name+'/af','w')
    f.write(nn.activation_function_name)
    f.close()
    mv=[]
    mv.append(nn.mean.flatten())
    mv.append(nn.var.flatten())
    mv=numpy.array(mv)
    numpy.savetxt(file_name+'/mv',mv)
    for i in range(0,nn.layer):
        numpy.savetxt(file_name+'/w'+str(i),nn.w[i])
        numpy.savetxt(file_name+'/b'+str(i),nn.b[i])
        
    print('保存成功')

In [34]:
def loadnn(file_name):
    para=parameter()
    f=numpy.loadtxt(file_name+'/parameter')
    para.input_nodes=int(f[0])
    para.layer=int(f[1])
    f=numpy.loadtxt(file_name+'/layer')
    para.layer_nodes=[]
    for i in range(0,para.layer):
        para.layer_nodes.append(int(f[i]))
    f=open(file_name+'/af')
    para.activation_function_name=f.read()
    f.close()
    nn=NeuralNetwork(para)
    for i in range(0,nn.layer):
        nn.w[i]=numpy.array(numpy.loadtxt(file_name+'/w'+str(i)),ndmin=2)
        nn.b[i]=numpy.array(numpy.loadtxt(file_name+'/b'+str(i)),ndmin=2)
    f=numpy.loadtxt(file_name+'/mv')
    nn.mean=f[0,:]
    nn.var=f[1,:]
    nn.mean=numpy.array(nn.mean,ndmin=2).T
    nn.var=numpy.array(nn.var,ndmin=2).T
    return nn

In [35]:
def obtain_train_set(path_1,path_0):
    tx=[]
    ty=[]
    
    path=path_1
    x_dirs=os.listdir(path)
    for x_file in x_dirs:
        fpath=os.path.join(path,x_file)
        file=Image.open(fpath).convert('L')
        x=file.resize((32,64))
        x=numpy.array(x)
        x=x.flatten()
        tx.append(x)
        ty.append([1])
        
    path=path_0
    x_dirs=os.listdir(path)
    for x_file in x_dirs:
        fpath=os.path.join(path,x_file)
        file=Image.open(fpath).convert('L')
        x=file.crop((0,0,320,480))
        x=x.resize((32,64))
        x=numpy.array(x)
        x=x.flatten()
        tx.append(x)
        ty.append([0])
        x=file.crop((320,0,640,480))
        x=x.resize((32,64))
        x=numpy.array(x)
        x=x.flatten()
        tx.append(x)
        ty.append([0])
    tx=numpy.array(tx)
    ty=numpy.array(ty)
    num=tx.shape[0]
    index=numpy.arange(num)
    numpy.random.shuffle(index)
    ttx=tx[index]
    tty=ty[index]
    tx=ttx[:num-1024,:]
    ty=tty[:num-1024,:]
    ttx=ttx[num-1024:,:]
    tty=tty[num-1024:,:]
    set=[tx,ty,ttx,tty]
    print('集合划分完成')
    return set
def obtain_car_train_set(path_1,path_0):
    tx=[]
    ty=[]
    
    path=path_1
    x_dirs=os.listdir(path)
    for x_file in x_dirs:
        fpath=os.path.join(path,x_file)
        file=Image.open(fpath).convert('L')
        x=file.resize((32,64))
        x=numpy.array(x)
        x=x.flatten()
        tx.append(x)
        ty.append([1])
        
    path=path_0
    x_dirs=os.listdir(path)
    for x_file in x_dirs:
        fpath=os.path.join(path,x_file)
        file=Image.open(fpath).convert('L')
        x=file.resize((32,64))
        x=numpy.array(x)
        x=x.flatten()
        tx.append(x)
        ty.append([0])
    tx=numpy.array(tx)
    ty=numpy.array(ty)
    num=tx.shape[0]
    index=numpy.arange(num)
    numpy.random.shuffle(index)
    ttx=tx[index]
    tty=ty[index]
    tx=ttx[:num-100,:]
    ty=tty[:num-100,:]
    ttx=ttx[num-100:,:]
    tty=tty[num-100:,:]
    set=[tx,ty,ttx,tty]
    print('集合划分完成')
    return set

In [21]:
#开始训练人的分类器
[train_x,train_y,test_x,test_y]=obtain_train_set('./data/Pedestrians/48x96','./data/NonPedestrians')

集合划分完成


In [28]:
para=parameter()
NN=NeuralNetwork(para)
error=error_test(NN,train_x.T,train_y.T,para)
error=error_test(NN,test_x.T,test_y.T,para)

背景误判率 99.57 %:（ 12945 / 13001 ）  对象误判率 0.03 %:（ 4 / 15123 ）
28124 个测试样本中错误 12949 个，错误率为 46.04 %.
背景误判率 99.79 %:（ 486 / 487 ）  对象误判率 0.0 %:（ 0 / 537 ）
1024 个测试样本中错误 486 个，错误率为 47.46 %.


In [29]:
NN=train(NN,train_x.T,train_y.T,test_x.T,test_y.T,para)
e=error_test(NN,train_x.T,train_y.T,para)

开始训练第  1  轮,每轮  7031 批
已完成 1 / 10  本轮用时 70.55 秒
背景误判率 7.39 %:（ 36 / 487 ）  对象误判率 11.36 %:（ 61 / 537 ）
1024 个测试样本中错误 97 个，错误率为 9.47 %.
开始训练第  2  轮,每轮  7031 批
已完成 2 / 10  本轮用时 72.07 秒
背景误判率 5.75 %:（ 28 / 487 ）  对象误判率 9.5 %:（ 51 / 537 ）
1024 个测试样本中错误 79 个，错误率为 7.71 %.
开始训练第  3  轮,每轮  7031 批
已完成 3 / 10  本轮用时 71.61 秒
背景误判率 4.72 %:（ 23 / 487 ）  对象误判率 8.38 %:（ 45 / 537 ）
1024 个测试样本中错误 68 个，错误率为 6.64 %.
开始训练第  4  轮,每轮  7031 批
已完成 4 / 10  本轮用时 71.26 秒
背景误判率 4.31 %:（ 21 / 487 ）  对象误判率 5.96 %:（ 32 / 537 ）
1024 个测试样本中错误 53 个，错误率为 5.18 %.
开始训练第  5  轮,每轮  7031 批
已完成 5 / 10  本轮用时 71.49 秒
背景误判率 4.31 %:（ 21 / 487 ）  对象误判率 6.15 %:（ 33 / 537 ）
1024 个测试样本中错误 54 个，错误率为 5.27 %.
开始训练第  6  轮,每轮  7031 批
已完成 6 / 10  本轮用时 75.07 秒
背景误判率 3.9 %:（ 19 / 487 ）  对象误判率 6.15 %:（ 33 / 537 ）
1024 个测试样本中错误 52 个，错误率为 5.08 %.
开始训练第  7  轮,每轮  7031 批
已完成 7 / 10  本轮用时 71.84 秒
背景误判率 3.7 %:（ 18 / 487 ）  对象误判率 4.84 %:（ 26 / 537 ）
1024 个测试样本中错误 44 个，错误率为 4.3 %.
开始训练第  8  轮,每轮  7031 批
已完成 8 / 10  本轮用时 67.31 秒
背景误判率 3.9 %:（ 19 / 487 ）

In [30]:
savenn(NN,'NNr')

保存成功


In [41]:
NN=loadnn('NNr')

In [42]:
#用于再训练
para=parameter()
para.epoch=1
para.learning_rate=0.001

error=error_test(NN,test_x.T,test_y.T,para)
error=error_test(NN,train_x.T,train_y.T,para)

NN=train(NN,train_x.T,train_y.T,test_x.T,test_y.T,para)
e=error_test(NN,train_x.T,train_y.T,para)

背景误判率 58.18 %:（ 32 / 55 ）  对象误判率 2.22 %:（ 1 / 45 ）
100 个测试样本中错误 33 个，错误率为 33.0 %.
背景误判率 59.1 %:（ 263 / 445 ）  对象误判率 2.58 %:（ 13 / 503 ）
948 个测试样本中错误 276 个，错误率为 29.11 %.
开始训练第  1  轮,每轮  237 批
已完成 1 / 1  本轮用时 2.32 秒
背景误判率 30.91 %:（ 17 / 55 ）  对象误判率 22.22 %:（ 10 / 45 ）
100 个测试样本中错误 27 个，错误率为 27.0 %.
背景误判率 30.56 %:（ 136 / 445 ）  对象误判率 14.91 %:（ 75 / 503 ）
948 个测试样本中错误 211 个，错误率为 22.26 %.


In [37]:
#开始训练车的分类器
[train_x,train_y,test_x,test_y]=obtain_car_train_set('./data/cars','./data/background')

集合划分完成


In [38]:
para=parameter()
para.layer=2
para.layer_nodes=[512,1]
para.learning_rate=0.01
para.activation_function_name='leaky_ReLU'
para.epoch=3
para.batchsize=4
b=NeuralNetwork(para)
error=error_test(b,train_x.T,train_y.T,para)
error=error_test(b,test_x.T,test_y.T,para)

背景误判率 95.73 %:（ 426 / 445 ）  对象误判率 3.78 %:（ 19 / 503 ）
948 个测试样本中错误 445 个，错误率为 46.94 %.
背景误判率 94.55 %:（ 52 / 55 ）  对象误判率 6.67 %:（ 3 / 45 ）
100 个测试样本中错误 55 个，错误率为 55.0 %.


In [39]:
b=train(b,train_x.T,train_y.T,test_x.T,test_y.T,para)
e=error_test(b,train_x.T,train_y.T,para)

开始训练第  1  轮,每轮  237 批
已完成 1 / 3  本轮用时 4.13 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
开始训练第  2  轮,每轮  237 批
已完成 2 / 3  本轮用时 4.87 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
开始训练第  3  轮,每轮  237 批
已完成 3 / 3  本轮用时 6.18 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
背景误判率 5.84 %:（ 26 / 445 ）  对象误判率 15.71 %:（ 79 / 503 ）
948 个测试样本中错误 105 个，错误率为 11.08 %.


In [40]:
savenn(b,'NNc')

保存成功
开始训练第  1  轮,每轮  237 批
已完成 1 / 3  本轮用时 4.17 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
开始训练第  2  轮,每轮  237 批
已完成 2 / 3  本轮用时 4.88 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
开始训练第  3  轮,每轮  237 批
已完成 3 / 3  本轮用时 4.83 秒
背景误判率 0.0 %:（ 0 / 55 ）  对象误判率 13.33 %:（ 6 / 45 ）
100 个测试样本中错误 6 个，错误率为 6.0 %.
背景误判率 5.39 %:（ 24 / 445 ）  对象误判率 15.31 %:（ 77 / 503 ）
948 个测试样本中错误 101 个，错误率为 10.65 %.
